# NMAPのInventoryモジュールの改良版でSNMP(161)ポートを自動ディスカバリして、snmp_factで機器の基本情報を自動収集して、ベンダーごとに情報を分別する

In [ ]:
#ansible.cfg
[defaults]
host_key_checking = False
#vault_password_file = ~/.rhv/vault-secret
fact_caching = yaml
fact_caching_connection = ./ansible_facts
cache_timeout = 3600

[inventory]
cache = yes
cache_connection = ./ansible_inventory
enable_plugins = nmap, advanced_host_list, yaml, ini

## addressでnmapしたいアドレスレンジを指定する

In [ ]:
#inventory
plugin: nmap
strict: True
address: XXX.XXX.XXX.XXX/ZZ
exclude: 
 - XXX.XXX.XXX.XXZ

In [ ]:
#play
name: nmap inventory plugin
hosts: all
gather_facts: false
connection: local

In [ ]:
#task
snmp_facts:
  host: "{{ item.value.ip }}"
  version: v2c
  community: XXXX
with_dict: "{{ hostvars }}"
register: result

In [ ]:
#task
set_fact:
  snmplist: "{{ result | json_query(query) }}"
vars:
  query: "results[].ansible_facts"
register: snmplist
run_once: true

In [ ]:
#task
set_fact:
  snmpresult: "{{ result | json_query(query) }}"
vars: 
  query: "results[].ansible_facts[][ansible_all_ipv4_addresses[?@ == `{{ inventory_hostname }}` ] | [0], ansible_sysobjectid]"
  

In [ ]:
#task
name: Execute Show command for Junos
cli_command:
  command: show chassis hardware
vars:
  ansible_connection: network_cli
  ansible_network_os: junos
  ansible_user: admin
  ansible_ssh_pass: admin
when: ( not item.0 == None ) and ("1.3.6.1.4.1.2636" in item.1 )
with_list: "{{ snmpresult }}"
register: junos

In [ ]:
#task
name: Execute Show command for IOSXR
cli_command:
  command: show inventory
vars:
  ansible_connection: network_cli
  ansible_network_os: iosxr
  ansible_user: admin
  ansible_ssh_pass: admin
when: ( not item.0 == None ) and ("1.3.6.1.4.1.9" in item.1 )
with_list: "{{ snmpresult }}"
register: iosxr

In [ ]:
#task
set_fact:
  junos_show: "{{  junos | json_query(query) | parse_cli_textfsm('/home/notebook/notebooks/ntc-templates/templates/juniper_junos_show_chassis_hardware.template')  }}"
vars: 
  query: "results[].stdout | [0]"

In [ ]:
#task
set_fact:
  iosxr_show: "{{  iosxr | json_query(query) | parse_cli_textfsm('/home/notebook/notebooks/ntc-templates/templates/cisco_ios_show_inventory.template')  }}"
vars: 
  query: "results[].stdout | [0]"

In [ ]:
#task
name: Execute Show command for Arista
cli_command:
  command: show inventory
vars:
  ansible_connection: network_cli
  ansible_network_os: eos
  ansible_user: admin
  ansible_ssh_pass: admin
when: ( not item.0 == None ) and ("1.3.6.1.4.1.30065" in item.1 )
with_list: "{{ snmpresult }}"
register: arista